In [1]:
import os
import librosa
import _pickle as cPickle
from scipy.io.wavfile import read
from pathlib import Path
import warnings
import numpy as np
from sklearn import preprocessing
import python_speech_features as mfcc
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
from sklearn import preprocessing
import python_speech_features as mfcc
#from speakerfeatures import extract_features

def calculate_delta(array):
    """Calculate and returns the delta of given feature vector matrix"""

    rows,cols = array.shape
    deltas = np.zeros((rows,20))
    N = 2
    for i in range(rows):
        index = []
        j = 1
        while j <= N:
            if i-j < 0:
                first = 0
            else:
                first = i-j
            if i+j > rows -1:
                second = rows -1
            else:
                second = i+j
            index.append((second,first))
            j+=1
        deltas[i] = ( array[index[0][0]]-array[index[0][1]] + (2 * (array[index[1][0]]-array[index[1][1]])) ) / 10
    return deltas

def extract_features(audio,rate):
    """extract 20 dim mfcc features from an audio, performs CMS and combines 
    delta to make it 40 dim feature vector"""    
    
    mfcc_feat = mfcc.mfcc(audio,rate, 0.025, 0.01,20,appendEnergy = True)
    
    mfcc_feat = preprocessing.scale(mfcc_feat)
    delta = calculate_delta(mfcc_feat)
    combined = np.hstack((mfcc_feat,delta)) 
    return combined

In [3]:
#dest = "..\\data\\mfcc1"  

train_file = "..\\data\\f_list\\wav_list_train.txt"    

test_file = "..\\data\\f_list\\wav_list_test.txt"  


file_paths = open(train_file,'r')

# Extracting features for each speaker (5 files per speakers)
def get_features(): 
    file_paths = open(train_file,'r')
    features_normalize = []
    total_feature = []
    features = np.asarray(())
    for path in file_paths:    
        path = path.strip()
        file_name=os.path.basename(path)
#         split_filename=file_name.split('.')
#         molecule_name=split_filename[0]
#         print(path + ' before')
        audio, sr1 = librosa.load(path, sr=44100)
        data=librosa.resample(audio, sr1, 16000)
#         picklefile =molecule_name+".mfcc"+".txt"
#         folder=Path(path).parts
#         b='AR'
        # extract 40 dimensional MFCC & delta MFCC features
        vector   = extract_features(data,16000)
#         print(path + ' middle1')
        if features.size == 0:
            features = vector
        #print(features)
#         if folder[3]==b:
#             full_path = os.path.join(dest+"\\AR",picklefile)
#             print(path + ' after1')
#             with open(full_path, 'w') as filehandle:
#                 for listitem in features:
#                     filehandle.write('%s\n' % listitem)
        
        else:
            features = np.vstack((features, vector))
#         #print(features)
#         if folder[3]==b:
#             full_path = os.path.join(dest+"\\AR",picklefile)
#             print(path + ' after1')
#             with open(full_path, 'w') as filehandle:
#                 for listitem in vector:
#                     filehandle.write('%s\n' % listitem)
#         else:
#             full_path1 = os.path.join(dest+"\\EN",picklefile)
#             print(path + ' after2')
#             with open(full_path1, 'w') as filehandle:
#                 for listitem in vector:
#                     filehandle.write('%s\n' % listitem)   

        total_feature.append(features)
    return total_feature

   
extracted_features = get_features() 
final_features = np.vstack(extracted_features)
print(final_features.shape)

(37474, 40)


In [4]:


# Extracting features for each speaker (5 files per speakers)
def get_label(): 
    file_paths = open(train_file,'r')
    features_normalize = []
    total_feature = []
    labels = []
    features = np.asarray(())
    
    for path in file_paths:    
        path = path.strip()
        file_name=os.path.basename(path)
#         split_filename=file_name.split('.')
#         molecule_name=split_filename[0]
#         print(path + ' before')
        audio, sr1 = librosa.load(path, sr=44100)
        data=librosa.resample(audio, sr1, 16000)
#         picklefile =molecule_name+".mfcc"+".txt"
#         folder=Path(path).parts
#         b='AR'
        # extract 40 dimensional MFCC & delta MFCC features
        vector   = extract_features(data,16000)
#         print(path + ' middle1')
        if features.size == 0:
            features = vector
        #print(features)
#         if folder[3]==b:
#             full_path = os.path.join(dest+"\\AR",picklefile)
#             print(path + ' after1')
#             with open(full_path, 'w') as filehandle:
#                 for listitem in features:
#                     filehandle.write('%s\n' % listitem)
        
        else:
            features = np.vstack((features, vector))
#         #print(features)
#         if folder[3]==b:
#             full_path = os.path.join(dest+"\\AR",picklefile)
#             print(path + ' after1')
#             with open(full_path, 'w') as filehandle:
#                 for listitem in vector:
#                     filehandle.write('%s\n' % listitem)
#         else:
#             full_path1 = os.path.join(dest+"\\EN",picklefile)
#             print(path + ' after2')
#             with open(full_path1, 'w') as filehandle:
#                 for listitem in vector:
#                     filehandle.write('%s\n' % listitem)

        file_name=os.path.basename(path)
         # print(file_name)
        split_filename=file_name.split('_')
        #print (split_filename)
        #split_filename1=file_name.split('_')[0]
        #print(split_filename1)
        molecule_name=split_filename[0]
        #fin_name=molecule_name.split('S')[1]
        #  print(fin_name)
        #final_name=fin_name.split('F')[0]
        #print(molecule_name)
        for r in range(features.shape[0]):
            #print(r)
            #print(path.split('\\')[0].split('-')[1])
            labels.append(molecule_name)
       
    return labels

   
label_value = get_label()
final_label = np.array(label_value)
print(final_label)
print(final_label.shape)

['S001F' 'S001F' 'S001F' ... 'S023M' 'S023M' 'S023M']
(37474,)


In [6]:
from sklearn.model_selection import train_test_split 

y = final_label   
x = preprocessing.StandardScaler().fit(final_features).transform(final_features.astype(float)) 

# as said before,  we will use 20% of the dataset for validation
validation_size = 0.20

# split the data into traning and testing
#X_train, X_test, y_train, y_test = train_test_split( x, y, test_size=validation_size, random_state=4)

print ('Train set:', x.shape,  y.shape)
#print ('Test set:', X_test.shape,  y_test.shape)

Train set: (37474, 40) (37474,)


In [7]:
#Tranining

from sklearn.neighbors import KNeighborsClassifier

k = 9 
neigh = KNeighborsClassifier(n_neighbors = k).fit(x ,y)

#The default metric is minkowski, and with p=2 is equivalent to the standard Euclidean metric.
print(neigh) 


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=9, p=2,
                     weights='uniform')


In [8]:
#Testing and Predicting
y_pred = neigh.predict(x)
print(y)

#y_pred = neigh.predict(X_test)
#print(y_pred)

['S001F' 'S001F' 'S001F' ... 'S023M' 'S023M' 'S023M']


In [ ]:
from sklearn import metrics

print("Train set Accuracy: ", metrics.accuracy_score(y_train, neigh.predict(X_train)))
#print("Test set Accuracy: ", metrics.accuracy_score(y_test, y_pred))